<a href="https://colab.research.google.com/github/Balaramansvs/NLP/blob/main/NLPBALA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import re
import math
import random
from collections import Counter, defaultdict
import pandas as pd

# -------------------
# STEP 1: Dataset
# -------------------
custom_text = """
In a quiet village, there lived an old clockmaker.
He spent his days repairing ancient timepieces and crafting new ones.
One evening, he discovered a mysterious golden gear inside a broken watch.
When he placed it into a clock, time began to flow backward.
The village soon found itself living yesterday all over again.
"""

# -------------------
# STEP 2: Preprocessing
# -------------------
def preprocess(text):
    text = text.lower()
    tokens = re.findall(r'\b[a-z]+\b', text)
    return tokens

tokens = preprocess(custom_text)

# Train/Test Split
random.seed(42)
split_point = int(0.8 * len(tokens))
train_tokens = tokens[:split_point]
test_tokens = tokens[split_point:]

# -------------------
# STEP 3: Build Models
# -------------------
def build_unigram_model(tokens):
    counts = Counter(tokens)
    total_count = sum(counts.values())
    return counts, total_count

def build_bigram_model(tokens):
    bigram_counts = Counter(zip(tokens[:-1], tokens[1:]))
    unigram_counts = Counter(tokens)
    return bigram_counts, unigram_counts

unigram_counts, total_unigrams = build_unigram_model(train_tokens)
bigram_counts, unigram_counts_for_bigram = build_bigram_model(train_tokens)
vocab = set(train_tokens)
vocab_size = len(vocab)

# -------------------
# STEP 4: Perplexity Functions
# -------------------
def perplexity_unigram(test_tokens, prob_func):
    N = len(test_tokens)
    log_prob_sum = 0
    for w in test_tokens:
        prob = prob_func(w)
        log_prob_sum += math.log(prob)
    return math.exp(-log_prob_sum / N)

def perplexity_bigram(test_tokens, prob_func):
    N = len(test_tokens) - 1
    log_prob_sum = 0
    for i in range(N):
        w1, w2 = test_tokens[i], test_tokens[i+1]
        prob = prob_func(w1, w2)
        log_prob_sum += math.log(prob)
    return math.exp(-log_prob_sum / N)

# -------------------
# STEP 5: Smoothing Probability Functions
# -------------------
# No smoothing
def prob_uni_no(w):
    return unigram_counts.get(w, 0) / total_unigrams if unigram_counts.get(w, 0) > 0 else 1e-8

def prob_bi_no(w1, w2):
    return bigram_counts.get((w1, w2), 0) / unigram_counts_for_bigram[w1] if bigram_counts.get((w1, w2), 0) > 0 else 1e-8

# Laplace (Add-1)
def prob_uni_laplace(w):
    return (unigram_counts.get(w, 0) + 1) / (total_unigrams + vocab_size)

def prob_bi_laplace(w1, w2):
    return (bigram_counts.get((w1, w2), 0) + 1) / (unigram_counts_for_bigram[w1] + vocab_size)

# Add-k
k_value = 0.5
def prob_uni_addk(w):
    return (unigram_counts.get(w, 0) + k_value) / (total_unigrams + k_value * vocab_size)

def prob_bi_addk(w1, w2):
    return (bigram_counts.get((w1, w2), 0) + k_value) / (unigram_counts_for_bigram[w1] + k_value * vocab_size)

# Good–Turing
bigram_count_of_counts = Counter(bigram_counts.values())
total_bigrams = sum(bigram_counts.values())
def prob_bi_goodturing(w1, w2):
    c = bigram_counts.get((w1, w2), 0)
    if c < max(bigram_count_of_counts.keys()):
        return ((c+1) * (bigram_count_of_counts.get(c+1, 0) / bigram_count_of_counts.get(c, 1))) / total_bigrams
    else:
        return c / total_bigrams

# Kneser–Ney
continuation_counts = Counter([w2 for (w1, w2) in bigram_counts.keys()])
def prob_bi_kneserney(w1, w2, discount=0.75):
    bigram_count = bigram_counts.get((w1, w2), 0)
    lambda_w1 = (discount / unigram_counts_for_bigram[w1]) * len([w for (u, w) in bigram_counts if u == w1])
    p_continuation = continuation_counts[w2] / sum(continuation_counts.values())
    return max(bigram_count - discount, 0) / unigram_counts_for_bigram[w1] + lambda_w1 * p_continuation

# Witten–Bell
unique_continuations = defaultdict(int)
for (w1, w2) in bigram_counts.keys():
    unique_continuations[w1] += 1

def prob_bi_wittenbell(w1, w2):
    T = unique_continuations[w1]
    Z = vocab_size - T
    bigram_count = bigram_counts.get((w1, w2), 0)
    if bigram_count > 0:
        return bigram_count / (unigram_counts_for_bigram[w1] + T)
    else:
        return T / (Z * (unigram_counts_for_bigram[w1] + T))

# -------------------
# STEP 6: Evaluate All Methods
# -------------------
results = []
methods = [
    ("No Smoothing", prob_uni_no, prob_bi_no),
    ("Laplace", prob_uni_laplace, prob_bi_laplace),
    ("Add-k", prob_uni_addk, prob_bi_addk),
    ("Good–Turing", prob_uni_no, prob_bi_goodturing), # Good–Turing not done for unigram here
    ("Kneser–Ney", prob_uni_no, prob_bi_kneserney),   # Kneser–Ney is bigram only
    ("Witten–Bell", prob_uni_no, prob_bi_wittenbell)  # Witten–Bell is bigram only
]

for name, uni_func, bi_func in methods:
    try:
        pp_uni = perplexity_unigram(test_tokens, uni_func)
    except:
        pp_uni = None
    try:
        pp_bi = perplexity_bigram(test_tokens, bi_func)
    except:
        pp_bi = None
    results.append([name, pp_uni, pp_bi])

# -------------------
# STEP 7: Show Table
# -------------------
df = pd.DataFrame(results, columns=["Method", "Unigram Perplexity", "Bigram Perplexity"])
print(df)


         Method  Unigram Perplexity  Bigram Perplexity
0  No Smoothing        2.637694e+07       1.000000e+08
1       Laplace        7.605340e+01       3.809884e+01
2         Add-k        1.122140e+02       3.819542e+01
3   Good–Turing        2.637694e+07       1.000000e+00
4    Kneser–Ney        2.637694e+07                NaN
5   Witten–Bell        2.637694e+07                NaN
